In [1]:
import torch
import numpy as np
from tqdm import tqdm
import data_utils
import argparse
import pandas as pd
import survface_helper
import sys, os
sys.path.insert(0, os.path.dirname(os.getcwd()))

import net
import scipy

/home/taes/anaconda3/envs/adaface/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ['CUDA_VISIBLE_DEVICES'] = '1'


In [3]:
def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

In [4]:
def l2_norm(input, axis=1):
    """l2 normalize
    """
    norm = torch.norm(input, 2, axis, True)
    output = torch.div(input, norm)
    return output, norm


In [5]:
def fuse_features_with_norm(stacked_embeddings, stacked_norms, fusion_method='norm_weighted_avg'):

    assert stacked_embeddings.ndim == 3 # (n_features_to_fuse, batch_size, channel)
    if stacked_norms is not None:
        assert stacked_norms.ndim == 3 # (n_features_to_fuse, batch_size, 1)
    else:
        assert fusion_method not in ['norm_weighted_avg', 'pre_norm_vector_add']

    if fusion_method == 'norm_weighted_avg':
        weights = stacked_norms / stacked_norms.sum(dim=0, keepdim=True)
        fused = (stacked_embeddings * weights).sum(dim=0)
        fused, _ = l2_norm(fused, axis=1)
        fused_norm = stacked_norms.mean(dim=0)
    elif fusion_method == 'pre_norm_vector_add':
        pre_norm_embeddings = stacked_embeddings * stacked_norms
        fused = pre_norm_embeddings.sum(dim=0)
        fused, fused_norm = l2_norm(fused, axis=1)
    elif fusion_method == 'average':
        fused = stacked_embeddings.sum(dim=0)
        fused, _ = l2_norm(fused, axis=1)
        if stacked_norms is None:
            fused_norm = torch.ones((len(fused), 1))
        else:
            fused_norm = stacked_norms.mean(dim=0)
    elif fusion_method == 'concat':
        fused = torch.cat([stacked_embeddings[0], stacked_embeddings[1]], dim=-1)
        if stacked_norms is None:
            fused_norm = torch.ones((len(fused), 1))
        else:
            fused_norm = stacked_norms.mean(dim=0)
    elif fusion_method == 'faceness_score':
        raise ValueError('not implemented yet. please refer to https://github.com/deepinsight/insightface/blob/5d3be6da49275602101ad122601b761e36a66a01/recognition/_evaluation_/ijb/ijb_11.py#L296')
        # note that they do not use normalization afterward.
    else:
        raise ValueError('not a correct fusion method', fusion_method)

    return fused, fused_norm


In [6]:
def infer(model, dataloader, use_flip_test, fusion_method):
    model.eval()
    features = []
    norms = []
    with torch.no_grad():
        for images, idx in tqdm(dataloader):

            feature = model(images.to("cuda:0"))
            if isinstance(feature, tuple):
                feature, norm = feature
            else:
                norm = None

            if use_flip_test:
                fliped_images = torch.flip(images, dims=[3])
                flipped_feature = model(fliped_images.to("cuda:0"))
                if isinstance(flipped_feature, tuple):
                    flipped_feature, flipped_norm = flipped_feature
                else:
                    flipped_norm = None

                stacked_embeddings = torch.stack([feature, flipped_feature], dim=0)
                if norm is not None:
                    stacked_norms = torch.stack([norm, flipped_norm], dim=0)
                else:
                    stacked_norms = None

                fused_feature, fused_norm = fuse_features_with_norm(stacked_embeddings, stacked_norms, fusion_method=fusion_method)
                features.append(fused_feature.cpu().numpy())
                norms.append(fused_norm.cpu().numpy())
            else:
                features.append(feature.cpu().numpy())
                norms.append(norm.cpu().numpy())

    features = np.concatenate(features, axis=0)
    norms = np.concatenate(norms, axis=0)
    return features, norms


In [7]:
def load_pretrained_model(model_name='ir50'):
    # load model and pretrained statedict
    ckpt_path = adaface_models[model_name][0]
    arch = adaface_models[model_name][1]

    model = net.build_model(arch)
    statedict = torch.load(ckpt_path)['state_dict']
    model_statedict = {key[6:]:val for key, val in statedict.items() if key.startswith('model.')}
    model.load_state_dict(model_statedict)
    model.eval()
    return model


In [8]:
def get_all_files(root, extension_list=['.jpg', '.png', '.jpeg']):

    all_files = list()
    for (dirpath, dirnames, filenames) in os.walk(root):
        all_files += [os.path.join(dirpath, file) for file in filenames]
    if extension_list is None:
        return all_files
    all_files = list(filter(lambda x: os.path.splitext(x)[1] in extension_list, all_files))
    return all_files



In [9]:
model_name = 'ir'
adaface_models={'ir50': ["../pretrained/ir50_casia_adaface_07131_38ep.ckpt", 'ir_50']}
model = load_pretrained_model(model_name)
model.cuda()

KeyError: 'ir'

In [385]:
model_name = 'ir101'
adaface_models={'ir101': ["../experiments/ir101_ufo_weight_15_time_cos_0502_11-01_0/last.ckpt", 'ir_101']}
model = load_pretrained_model(model_name)
model.cuda()

Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (body): Sequential(
    (0): BasicBlockIR(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1

In [298]:
model_name = 'ir101'
adaface_models={'ir101': ["../experiments/ir101_ms1mv2_adaface_02-08_0/last.ckpt", 'ir_101']}
model = load_pretrained_model(model_name)
model.cuda()

Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (body): Sequential(
    (0): BasicBlockIR(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1

In [ ]:
'./'

In [10]:
model_name = 'ir50'
adaface_models={'ir50': ["../experiments/ir50_casia_cosw15_11-15_2/epoch=24-step=23974.ckpt", 'ir_50']}
model = load_pretrained_model(model_name)
model.cuda()

Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (body): Sequential(
    (0): BasicBlockIR(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1

In [253]:
model_name = 'ir50'
adaface_models={'ir50': ["../experiments/ir50_casia_cosw_11-15_3/epoch=26-step=25892.ckpt", 'ir_50']}
model = load_pretrained_model(model_name)
model.cuda()

Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (body): Sequential(
    (0): BasicBlockIR(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1

In [210]:
model_name = 'ir50'
adaface_models={'ir50': ["../experiments/ir50_casia_arc_11-15_4/epoch=27-step=26851.ckpt", 'ir_50']}
model = load_pretrained_model(model_name)
model.cuda()

Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (body): Sequential(
    (0): BasicBlockIR(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1

In [342]:
model_name = 'ir50'
adaface_models={'ir50': ['../pretrained/ir50_casia_adaface_07131_38ep.ckpt', 'ir_50']}
model = load_pretrained_model(model_name)
model.cuda()

Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (body): Sequential(
    (0): BasicBlockIR(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1

In [121]:
model_name = 'ir50'
adaface_models={'ir50': ['../experiments/ir50_ufo_weight15_time_cos_0502_10-28_0/epoch=38-step=37400.ckpt', 'ir_50']}
model = load_pretrained_model(model_name)
model.cuda()

Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (body): Sequential(
    (0): BasicBlockIR(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1

In [10]:
model_name = 'ir50'
adaface_models={'ir50': ['../experiments/ir50_cosw_weight20_time_cos_0502_12-04_0/epoch=36-step=35482.ckpt', 'ir_50']}
model = load_pretrained_model(model_name)
model.cuda()

Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (body): Sequential(
    (0): BasicBlockIR(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1

In [11]:
'../../face_data/surv_ver_origin/verification_images/'

'../../face_data/surv_ver_origin/verification_images/'

In [12]:
alignment_dir_name = 'surv_ver_origin/verification_images'

In [13]:
suvface_root = '../../face_data/'

In [14]:
positive_dict = scipy.io.loadmat(os.path.join(suvface_root, 'QMUL-SurvFace/Face_Verification_Test_Set/positive_pairs_names.mat'))

In [15]:
negative_dict = scipy.io.loadmat(os.path.join(suvface_root, 'QMUL-SurvFace/Face_Verification_Test_Set/negative_pairs_names.mat'))

In [16]:
positive_dict

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Tue Jun  5 15:37:39 2018',
 '__version__': '1.0',
 '__globals__': [],
 'positive_pairs_names': array([[array(['4088_cam2_1.jpg'], dtype='<U15'),
         array(['4088_cam3_1.jpg'], dtype='<U15')],
        [array(['9713_cam2_58.jpg'], dtype='<U16'),
         array(['9713_cam3_86.jpg'], dtype='<U16')],
        [array(['4541_cam1_1.jpg'], dtype='<U15'),
         array(['4541_cam2_4.jpg'], dtype='<U15')],
        ...,
        [array(['7055_cam2_1.jpg'], dtype='<U15'),
         array(['7055_cam3_3.jpg'], dtype='<U15')],
        [array(['8010_cam2_1.jpg'], dtype='<U15'),
         array(['8010_cam2_3.jpg'], dtype='<U15')],
        [array(['4975_cam2_1.jpg'], dtype='<U15'),
         array(['4975_cam2_2.jpg'], dtype='<U15')]], dtype=object)}

align 이미지 path
pos랑 neg 구별 
label...? 

In [17]:
pos1_paths = [os.path.join(suvface_root, alignment_dir_name, p[0].item()) for p in positive_dict['positive_pairs_names']]
pos2_paths = [os.path.join(suvface_root, alignment_dir_name, p[1].item()) for p in positive_dict['positive_pairs_names']]


neg1_paths = [os.path.join(suvface_root, alignment_dir_name, p[0].item()) for p in negative_dict['negative_pairs_names']]
neg2_paths = [os.path.join(suvface_root, alignment_dir_name, p[1].item()) for p in negative_dict['negative_pairs_names']]

In [18]:
image_paths = get_all_files(os.path.join(suvface_root, alignment_dir_name))

In [19]:
image_paths

['../../face_data/surv_ver_origin/verification_images/3149_cam1_1.jpg',
 '../../face_data/surv_ver_origin/verification_images/3634_cam1_1.jpg',
 '../../face_data/surv_ver_origin/verification_images/6248_cam1_1.jpg',
 '../../face_data/surv_ver_origin/verification_images/588_cam2_1.jpg',
 '../../face_data/surv_ver_origin/verification_images/48_cam3_1.jpg',
 '../../face_data/surv_ver_origin/verification_images/7624_cam3_4.jpg',
 '../../face_data/surv_ver_origin/verification_images/200_cam2_1.jpg',
 '../../face_data/surv_ver_origin/verification_images/2188_cam2_1.jpg',
 '../../face_data/surv_ver_origin/verification_images/4316_cam1_1.jpg',
 '../../face_data/surv_ver_origin/verification_images/1711_cam2_1.jpg',
 '../../face_data/surv_ver_origin/verification_images/3805_cam1_1.jpg',
 '../../face_data/surv_ver_origin/verification_images/1067_cam2_1.jpg',
 '../../face_data/surv_ver_origin/verification_images/7457_cam2_3.jpg',
 '../../face_data/surv_ver_origin/verification_images/8574_cam2_10.j

In [20]:
def get_key(image_path):
        return os.path.splitext(os.path.basename(image_path))[0]

In [21]:
os.path.splitext(os.path.basename(image_paths[0]))

('3149_cam1_1', '.jpg')

In [22]:
get_key(image_paths[0])

'3149_cam1_1'

In [23]:
index_dict = {}
for i, image_path in enumerate(image_paths):
    index_dict[get_key(image_path)] = i

In [24]:
indices_pos1 = np.array([index_dict[get_key(img)] for img in pos1_paths])
indices_pos2 = np.array([index_dict[get_key(img)] for img in pos2_paths])

In [25]:
indices_neg1 = np.array([index_dict[get_key(img)] for img in neg1_paths])
indices_neg2 = np.array([index_dict[get_key(img)] for img in neg2_paths])

In [26]:
indices_pos1

array([6724, 1625, 9330, ..., 9914, 7500, 6646])

In [27]:
FARs = [1e-3, 1e-2, 1e-1, 3e-1] 

In [28]:
# adaface_models={'ir50': ["../pretrained/ir50_casia_adaface_07131_38ep.ckpt", 'ir_50']}
# # adaface_models={'ir50': ["../experiments/ir50_ufo_weight15_time_cos_0502_10-28_0/epoch=38-step=37400.ckpt", 'ir_50']}


# model = load_pretrained_model('ir50')
# model.cuda()


In [29]:
dataloader = data_utils.prepare_dataloader(image_paths, 128, num_workers=0)

In [30]:
features, norms = infer(model, dataloader, use_flip_test=True, fusion_method='pre_norm_vector_add')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:15<00:00,  4.94it/s]


In [31]:
features.shape

(10051, 512)

In [32]:
def inner_product(x1, x2):
    return np.sum(np.einsum('ik,ik->ik', x1, x2),-1)


In [33]:
feat_pos1 = features[indices_pos1]
feat_pos2 = features[indices_pos2]

In [34]:
score_mat_p = np.sum(np.einsum('ik,ik->ik', feat_pos1, feat_pos2),-1)

In [35]:
score_mat_p

array([0.52606255, 0.76892823, 0.22668894, ..., 0.7946373 , 0.7874795 ,
       0.5535009 ], dtype=float32)

In [36]:
feat_neg1 = features[indices_neg1]
feat_neg2 = features[indices_neg2]

In [37]:
score_mat_n = np.sum(np.einsum('ik,ik->ik', feat_neg1, feat_neg2),-1)

In [38]:
label_p = np.ones(score_mat_p.shape,dtype=bool)

In [39]:
label_n = np.zeros(score_mat_n.shape,dtype=bool)

In [40]:
score_mat = np.concatenate([score_mat_p,score_mat_n])

In [41]:
label_mat = np.concatenate([label_p,label_n])

In [42]:
label_mat.dtype

dtype('bool')

In [43]:
score_neg = score_mat[~label_mat]

In [44]:
score_neg[::-1].sort()

In [45]:
FARs = np.array(FARs)

In [46]:
num_neg = len(score_neg)

In [47]:
num_false_alarms = np.round(num_neg * FARs).astype(np.int32)

In [48]:
thresholds = []

In [49]:
epsilon=1e-8

In [50]:
for num_false_alarm in num_false_alarms:
    if num_false_alarm==0:
        threshold = score_neg[0] + epsilon
    else:
        threshold = score_neg[num_false_alarm-1]
        thresholds.append(threshold)
thresholds = np.array(thresholds)

In [51]:
thresholds

array([0.89658135, 0.85635686, 0.7764789 , 0.673638  ], dtype=float32)

In [52]:
TARs = np.zeros(thresholds.shape[0])
FARs = np.zeros(thresholds.shape[0])
false_accept_indices = []
false_reject_indices = []

for i,threshold in enumerate(thresholds):
    accept = score_mat >= threshold
    TARs[i] = np.mean(accept[label_mat])
    FARs[i] = np.mean(accept[~label_mat])

In [53]:
TARs

array([0.02330827, 0.07575188, 0.27161654, 0.51898496])

In [341]:
array([0.03157895, 0.09191729, 0.34511278, 0.59022556])

NameError: name 'array' is not defined

In [ ]:
#adaface
#0.00451128, 0.0387218 , 0.18289474, 0.4137218

In [ ]:
# cosw 10
# 0.00451128, 0.02424812, 0.17462406, 0.41353383

In [ ]:
# cosw 15
# 0.00451128, 0.02424812, 0.17462406, 0.41353383

In [ ]:
# cos 
# 0.00827068, 0.03101504, 0.1693609 , 0.37556391

In [ ]:
TARs

In [ ]:
# ada last 0.00676692, 0.03759398, 0.30112782, 0.5674812 

In [58]:
## array([0.01259398, 0.07387218, 0.30413534, 0.57086466])


In [197]:
FARs

array([0.00093985, 0.00996241, 0.1       , 0.3       ])